In [9]:
import gym
from gym import wrappers
import numpy as np

In [16]:
for i in gym.envs.registry:
    #print(i)

SyntaxError: incomplete input (159489049.py, line 2)

In [117]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size):
        self.learning_rate = 0.005
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))
        
    def forward(self, observation):
        self.hidden_in = np.dot(self.W1, observation) + self.b1
        self.hidden_out = (self.hidden_in > 0) * self.hidden_in
        self.output_in = np.dot(self.W2, self.hidden_out) + self.b2
        self.output_out = self.softmax(self.output_in)
        return self.output_out
    
    def backprop(self, observation, Error):
        reluOutput = np.where(self.output_in > 0, 1, 0)    
        delta2 = reluOutput * Error

        dw2 = np.outer(delta2, self.hidden_out)
        db2 = delta2

        reluInput = np.where(self.hidden_in > 0, 1, 0)
        delta1 = reluInput * np.dot(self.W2.T, delta2)

        dw1 = np.outer(delta1, observation.reshape(-1, 1))
        db1 = delta1

        print("Shape of dw1:", dw1.shape)
        print("Shape of self.W1:", self.W1.shape)
        print("Shape of observation:", observation.reshape(-1, 1).shape)
        print("sjape of delta 1:", delta1.shape)
        # Update Rules
        self.W1 += self.learning_rate * dw1
        self.W2 += self.learning_rate * dw2
        self.b1 += self.learning_rate * db1
        self.b2 += self.learning_rate * db2  

    def softmax(self, Z):
        e_Z = np.exp(Z - np.max(Z, axis=0))
        return e_Z / np.sum(e_Z, axis=0)


In [118]:
epsilon_0 = 0.6 
beta = 0.00005
gamma = 0.85  
eta = 0.005 

env = gym.make('CartPole-v1', render_mode="human")
observation = env.reset()

s_size = env.observation_space.shape[0]
a_size = env.action_space.n
hidden_size = 8

net = ANN(s_size, hidden_size, a_size)

done = False
while not done:
    env.render()
    
    probabilities = net.forward(observation[0])
    averages = np.mean(probabilities, axis=1)
    action = np.argmax(averages)
    
    observation_next, reward, done, info, _ = env.step(action)
    Error = reward - np.max(averages)
    net.backprop(observation[0], Error)
    
env.close()

Shape of dw1: (64, 4)
Shape of self.W1: (8, 4)
Shape of observation: (4, 1)
sjape of delta 1: (8, 8)


ValueError: operands could not be broadcast together with shapes (8,4) (64,4) (8,4) 

: 